In [1]:
import requests
import json, re, time
from datetime import datetime
import pandas as pd

In [2]:
with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json',
            'cookie': 'ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip; ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version"
    ]

In [3]:
def getTestSettings(cc, url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc
                      }
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

### StartPageLoadTest function

In [4]:
def startPageLoadTest(test_settings):
    
    test_url = API_ENDPOINT + "StartPageLoadTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartPageLoadTestResult']['Status']['StatusText']):
        return res['StartPageLoadTestResult']['TestID']
    else:
        return "FAILED"

### retrievePageLoadTestResults function

In [5]:
def retrievePageLoadTestResults(testID):
    
    url = API_ENDPOINT + "GetPageLoadResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['PageLoadTestResults']

## Retrieve results

In [6]:
def getResultsFromTestIDs(df_tests_url):
    df_results = pd.DataFrame(columns=['domain',
                                       'cc',
                                       'ProbeInfo',
                                       'TestDateTime',
                                       'StatusCode',
                                       'StatusText',
                                       'DNSLookupTime', 
                                       'Destination', 
                                       'HTTPStatus', 
                                       'InitialConnection', 
                                       'NumberOfRequests', 
                                       'PageLoadTime', 
                                       'SSLNegotiationTime', 
                                       'TTFB', 
                                       'TotalDownloadedBytes'
                                      ])
    
    #Exclude FAILED items
    for index, row in df_tests_url.loc[df_tests_url.testID!='FAILED'].iterrows():
        
        domain = row['domain']
        cc = row['cc']
        testID = row['testID']
        pageLoadTestResults =  retrievePageLoadTestResults(testID)

        if (pageLoadTestResults is None or len(pageLoadTestResults) < 1):
            continue
        
        for res in pageLoadTestResults:
        
            probeInfo = res['ProbeInfo']
            testDateTime = res['TestDateTime']

            #extract the epoch
            m = re.search('/Date\((\d+)\+0000\)/', testDateTime)
            if m:
                testDateTime = m.group(1)[:-3]

            #convert into human readable format
            testDateTime = datetime.fromtimestamp(int(testDateTime)).strftime('%Y-%m-%d')
            statusCode = res['TestStatus']['StatusCode']
            statusText = res['TestStatus']['StatusText']
            dnsLookupTime = res['DNSLookupTime']
            destination =  res['Destination']
            hTTPStatus = res['HTTPStatus']
            initialConnection = res['InitialConnection']
            numberOfRequests =  res['NumberOfRequests'] 
            pageLoadTime =  res['PageLoadTime'] 
            sslNegotiationTime = res['SSLNegotiationTime'] 
            ttfb = res['TTFB']
            totalDownloadedBytes = res['TotalDownloadedBytes']

            df_results = df_results.append({
                'domain': domain,
                'cc': cc,
                'ProbeInfo':probeInfo,
                'TestDateTime': testDateTime,
                'StatusCode':statusCode,
                'StatusText':statusText,
                'DNSLookupTime':dnsLookupTime,
                'Destination':destination,
                'HTTPStatus':hTTPStatus,
                'InitialConnection':initialConnection,
                'NumberOfRequests':numberOfRequests,
                'PageLoadTime':pageLoadTime,
                'SSLNegotiationTime':sslNegotiationTime,
                'TTFB':ttfb,
                'TotalDownloadedBytes':totalDownloadedBytes
            }, ignore_index=True)

    return df_results

In [7]:
def runMeasurements(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettings(cc, url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    
    

In [8]:
df_links = pd.read_csv('data/links_sample.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [9]:
#runMeasurements(df_links, 'ori_amp_url', 'data/ori_amp_url_testID_4.csv')
runMeasurements(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID.csv')

9932c6c6-bc52-4ae9-9fbc-6fbbf9a5cfdc
db49d6e8-da6f-4d45-b856-958d02f1559e
2a0172b8-cc5b-4c3c-8848-5dcf3a9c4ede
39000685-e740-4e21-afac-d3e0cdce4cdd
22ed06bf-2381-4971-9368-2ba622a4bf61
699d5ace-aadd-4930-beac-b1f42089ed00
ab4c4e74-61e1-4a8f-978d-c194d69c9c06
47d51a35-cb65-4b82-8e56-f50c70f44b8e
b328290b-bd86-43e0-b7da-2a39e4d69284
801ddc86-6dfd-4769-8b29-dd11c8570cb6
71f09bb8-59fc-49d9-bf26-63b33871e54f
ee8bb725-8e15-4b68-8a18-b1d5da46c32a
98d84152-7449-4065-b011-04ca0634dd33
cdfab357-2693-4595-b44c-88b6f0e94c2d
3675c13e-2b98-40f0-8c71-427bd30888ed
e9ac4109-1f8b-4a30-be0a-bc5f79304f16
dfe5581a-e88c-48c9-9e3a-23acb0a26840
1a86d142-b256-4f6c-85fd-809c609d3091
09946563-a397-49e1-aaba-07e4256e7c80
d39538f9-01ac-4848-9fe9-3e49d73902db
a34aa753-72f8-4ed8-8940-493985231035
eb5e3990-a467-406a-946f-94ce84184234
8f684841-ac92-4f6e-a042-004d12ad0fdf
e331b284-f5cc-4f6a-a934-7d568ecaa29f
c8490772-89b8-4e05-920b-7399aeaec979
04d9b655-2a6b-4a95-8663-c15325799e94
195e6e48-7502-493c-8fd0-8ec2e57f0bb0
3

In [50]:
#df_testIDs_ori_amp = pd.read_csv('data/ori_amp_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [51]:
#df_results_ori_amp = getResultsFromTestIDs(df_testIDs_ori_amp)

In [52]:
#df_results_ori_amp.to_csv("data/ori_amp_url_results.csv", sep='|', encoding='utf-8', index=False)    